In [86]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
from tensorflow import keras

import pandas as pd
import numpy as np

In [87]:
train_df = pd.read_csv('./data/iris.csv')
np.random.shuffle(train_df.values)

print(train_df.head())

train_x = np.column_stack((train_df.SepalLengthCm.values, train_df.SepalWidthCm.values, train_df.PetalLengthCm.values, train_df.PetalWidthCm.values))

   Id  SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm      Species
0   1            5.1           3.5            1.4           0.2  Iris-setosa
1   2            4.9           3.0            1.4           0.2  Iris-setosa
2   3            4.7           3.2            1.3           0.2  Iris-setosa
3   4            4.6           3.1            1.5           0.2  Iris-setosa
4   5            5.0           3.6            1.4           0.2  Iris-setosa


In [88]:
print(train_df.Species.unique())

species_dict = {'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}

train_df['Species'] = train_df.Species.apply(lambda x: species_dict[x])

print(train_df.Species.unique())

['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']
[0 1 2]


In [92]:
model = keras.Sequential([
	keras.layers.Dense(64, input_shape=(4,), activation='relu'),
    # keras.layers.Dropout(0.2),									# % of neurons (of previous layer) that will be dropped
    keras.layers.Dense(64, activation='relu'),
	keras.layers.Dense(64, activation='relu'),						# adding layer, because quadratic example is more complex
	keras.layers.Dense(3, activation='sigmoid')])

model.compile(optimizer='adam', 
	          loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
	          metrics=['accuracy'])

results = model.fit(train_x, train_df.Species.values, batch_size=4, epochs=10)

# losses = results.history['loss']
# accuracies = results.history['accuracy']

Epoch 1/10


/home/hubert/.local/lib/python3.10/site-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


38/38 [==============================] - 1s 6ms/step - loss: 0.9865 - accuracy: 0.7267
Epoch 2/10
38/38 [==============================] - 0s 6ms/step - loss: 0.6426 - accuracy: 0.7867
Epoch 3/10
38/38 [==============================] - 0s 5ms/step - loss: 0.4076 - accuracy: 0.9267
Epoch 4/10
38/38 [==============================] - 0s 6ms/step - loss: 0.3077 - accuracy: 0.9200
Epoch 5/10
38/38 [==============================] - 0s 6ms/step - loss: 0.2223 - accuracy: 0.9533
Epoch 6/10
38/38 [==============================] - 0s 5ms/step - loss: 0.1677 - accuracy: 0.9600
Epoch 7/10
38/38 [==============================] - 0s 6ms/step - loss: 0.1473 - accuracy: 0.9533
Epoch 8/10
38/38 [==============================] - 0s 6ms/step - loss: 0.1536 - accuracy: 0.9467
Epoch 9/10
38/38 [==============================] - 0s 6ms/step - loss: 0.1330 - accuracy: 0.9533
Epoch 10/10
38/38 [==============================] - 0s 6ms/step - loss: 0.1249 - accuracy: 0.9467


In [93]:
test_df = pd.read_csv('./data/iris.csv')
test_x = np.column_stack((test_df.SepalLengthCm.values, test_df.SepalWidthCm.values, test_df.PetalLengthCm.values, test_df.PetalWidthCm.values))

test_df['Species'] = test_df.Species.apply(lambda x: species_dict[x])

print("EVALUATION")
evaluation = model.evaluate(test_x, test_df.Species.values)

EVALUATION
5/5 [==============================] - 0s 5ms/step - loss: 0.0879 - accuracy: 0.9733


/home/hubert/.local/lib/python3.10/site-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
